# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
import pandas as pd
import geopandas as gpd
import geoviews as gv
from holoviews import opts
import holoviews as hv

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\awais\Desktop\Statistical test\New folder\Starter_Code\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.57,87,100,2.24,NZ,1727811278
1,1,revda,56.8010,59.9303,2.48,69,0,1.28,RU,1727811279
2,2,abrantes,39.4667,-8.2000,21.18,73,100,2.24,PT,1727811155
3,3,holualoa,19.6228,-155.9522,27.53,65,0,2.57,US,1727811281
4,4,nema,57.5067,50.5011,3.73,54,0,2.75,RU,1727811282


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display
# Convert to GeoDataFrame using Lat and Lon
gdf = gpd.GeoDataFrame(city_data_df, geometry=gpd.points_from_xy(city_data_df.Lng, city_data_df.Lat))
# Create a geoView Points object with humidity controlling the point size
city_map = gv.Points(gdf, vdims=["City", "Humidity"])

# Customize the map
city_map.opts(
    opts.Points(
        size=hv.dim('Humidity')*0.2,  # Size of the points determined by the humidity
        color="blue",  # Optional: Change the color
        cmap="coolwarm",  # Optional: Change the color map
        colorbar=True,  # Display a color bar
        line_color="black",  # Optional: Add border color around points
        width=800, height=600,  # Size of the plot
        tools=['hover'],  # Enable hover tool to display city and humidity info
        title="Cities with Humidity Levels"
    )
)

# Display the map
city_map

# Display the map
# YOUR CODE HERE

:Points   [Longitude,Latitude]   (City,Humidity)

### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,87,
1,revda,RU,56.8010,59.9303,69,
2,abrantes,PT,39.4667,-8.2000,73,
3,holualoa,US,19.6228,-155.9522,65,
4,nema,RU,57.5067,50.5011,54,
...,...,...,...,...,...,...
583,nadym,RU,65.5333,72.5167,97,
584,malango,ID,-2.9627,119.9001,92,
585,frontera,MX,26.9333,-101.4500,41,
586,ardestan,IR,33.3761,52.3694,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
import requests
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters
geoapify_key = '5c69cae8319144d2800f5fe75b5abc00'  # Replace with your Geoapify API key

# Define parameters for the API request
params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',  # Hotel category
    'radius': radius,
    'limit': 1  # Get only the first hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found."
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    #print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name
0,waitangi,NZ,-43.9535,-176.5597,87,Hotel Chathams
1,revda,RU,56.8010,59.9303,69,Металлург
2,abrantes,PT,39.4667,-8.2000,73,Luna Hotel Turismo
3,holualoa,US,19.6228,-155.9522,65,Kona Hotel
4,nema,RU,57.5067,50.5011,54,No hotel found
...,...,...,...,...,...,...
583,nadym,RU,65.5333,72.5167,97,Янтарная
584,malango,ID,-2.9627,119.9001,92,Madarana
585,frontera,MX,26.9333,-101.4500,41,Quirino
586,ardestan,IR,33.3761,52.3694,26,Tourism Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display
gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df.Lng, hotel_df.Lat))
# Create a geoView Points object with humidity controlling the point size
city_map = gv.Points(gdf, vdims=["City", "Humidity","Country","Hotel Name"])

# Customize the map
city_map.opts(
    opts.Points(
        size=hv.dim('Humidity')*0.2,  # Size of the points determined by the humidity
        color="blue",  # Optional: Change the color
        cmap="coolwarm",  # Optional: Change the color map
        colorbar=True,  # Display a color bar
        line_color="black",  # Optional: Add border color around points
        width=800, height=600,  # Size of the plot
        tools=['hover'],  # Enable hover tool to display city and humidity info
        title="Cities with Humidity Levels"
    )
)

# Display the map
city_map

:Points   [Longitude,Latitude]   (City,Humidity,Country,Hotel Name)